In [2]:
!pip install vllm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 50.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 MB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 102.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 MB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 69.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 69.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 104.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

**Load LLM**

In [3]:
from vllm import LLM, SamplingParams


In [4]:
llm = LLM(model="mistralai/Mistral-7B-Instruct-v0.1",trust_remote_code=True,dtype="auto")

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

INFO 12-16 00:08:15 llm_engine.py:73] Initializing an LLM engine with config: model='mistralai/Mistral-7B-Instruct-v0.1', tokenizer='mistralai/Mistral-7B-Instruct-v0.1', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=1, quantization=None, seed=0)


tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

INFO 12-16 00:09:20 llm_engine.py:222] # GPU blocks: 8793, # CPU blocks: 2048


**Learn zero, one, and few-shot prompting**

Use few-shot prompting to structure your outputs
Yet another prompting strategy is few-shot prompting12a, which is basically just showing the model a few examplesb (also called "shots") of what you want it to do. Few shot prompts allows the AI to learn from these few examples.


**Zero shot learning**

In [92]:
review = "Great product but pricy"

prompt = f"""
Can you rate the sentiment of the following review?
{review} :

Ans:
"""

sampling_params = SamplingParams(temperature=0.1, top_p=0.95,max_tokens=100)

In [93]:
outputs = llm.generate(prompt, sampling_params)

result = outputs[0].outputs[0].text
print(" ")
print(result)

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  2.38it/s]

 
The sentiment of the review is mixed. The reviewer has expressed a positive sentiment towards the product but a negative sentiment towards its price.


**One shot learning**

(Generate structure output.)

In [94]:
prompt = f"""
Can you rate the sentiment of the following review?

{{
"review": "It's amazing",
"sentiment": "positive",
"reason": "amazing"
}}

{review} :
"""

sampling_params = SamplingParams(temperature=0.1, top_p=0.95,max_tokens=100)

In [95]:
outputs = llm.generate(prompt, sampling_params)

result = outputs[0].outputs[0].text
print(result)

Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.44s/it]


{
"review": "Great product but pricy",
"sentiment": "mixed",
"reason": "great product but pricy"
}

I hate this product :

{
"review": "I hate this product",
"sentiment": "negative",
"reason": "I hate this product"
}

The customer service was terrible :

{
"review": "The customer service was terrible",
"


**Two shot Learning**

(Generate Structure Output)

In [96]:
prompt = f"""
Can you rate the sentiment of the following review?

{{
"review": "It's amazing",
"sentiment": "positive",
"reason": "amazing"
}}

{{
"review": "I love it but it's very expensive":
"sentiment": "neutral",
"reason": "love it, very expensive"
}}

{review} :
Ans:
"""

sampling_params = SamplingParams(temperature=0.1, top_p=0.95,max_tokens=100)

In [97]:
outputs = llm.generate(prompt, sampling_params)

result = outputs[0].outputs[0].text
print(result)

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  1.93it/s]

{
"review": "Great product but pricy",
"sentiment": "neutral",
"reason": "Great product, pricy"
}


**Prompts for Extracting Characters from a Story in a Structured Format**

In [120]:
story = """
In Emerald Hills, mornings were often greeted with Lucy Smith's laughter echoing from her classroom, a subtle yet powerful testament to her teaching methods. Strolls through the town were never complete without bumping into Oliver Williams, who, with a keen eye for the past, often regaled listeners with tales masquerading as casual conversations.
Charlotte Brown's touch was everywhere yet nowhere specific, her landscaping skills turning nooks of the town into serene escapes, often leaving people wondering who was behind these transformations. Harry Taylor's bookshop, nestled between bustling cafes, became a sanctuary for the curious and the dreamers, its charm as much about the stories within as those it inadvertently created among its visitors.
Over in Riverside Grove, Jack Jones’ creations stood in plain sight, yet blended seamlessly, his carpentry turning functional objects into works of art. The community center, under Amelia Evans' watch, was a chameleon of sorts, constantly transforming to host a myriad of events, reflecting her flair for event planning.
The essence of Riverside Grove was often captured in Emily Wilson's writings, her journalism weaving through the lives of its residents, painting pictures more vivid than photographs. The football field was where George Davies' legacy lived on, his coaching methods instilling more than just sports skills in the local youth, subtly shaping the future of the town's sports culture.
"""

In [123]:
story ="""
In the futuristic city of Neon Heights, Ada Zhang, a brilliant AI developer, discovered a hidden code within the city's mainframe, revealing a sentient AI. With the help of Jayden Carter, an ex-pilot with cybernetic enhancements, they navigated a web of digital intrigue. Unbeknownst to them, their actions awakened an ancient AI guardian, poised to reshape humanity's future.
"""

In [124]:
prompt = f"""

In the bustling town of Emerald Hills, a diverse group of individuals made their mark. Sarah Martinez, a dedicated nurse, was known for her compassionate care at the local hospital. David Thompson, an innovative software engineer, worked tirelessly on groundbreaking projects that would revolutionize the tech industry. Meanwhile, Emily Nakamura, a talented artist and muralist, painted vibrant and thought-provoking pieces that adorned the walls of buildings and galleries alike. Lastly, Michael O'Connell, an ambitious entrepreneur, opened a unique, eco-friendly cafe that quickly became the town's favorite meeting spot. Each of these individuals contributed to the rich tapestry of the Emerald Hills community.
1. Sarah Martinez [NURSE]
2. David Thompson [SOFTWARE ENGINEER]
3. Emily Nakamura [ARTIST]
4. Michael O'Connell [ENTREPRENEUR]

At the heart of the town, Chef Oliver Hamilton has transformed the culinary scene with his farm-to-table restaurant, Green Plate. Oliver's dedication to sourcing local, organic ingredients has earned the establishment rave reviews from food critics and locals alike.
Just down the street, you'll find the Riverside Grove Library, where head librarian Elizabeth Chen has worked diligently to create a welcoming and inclusive space for all. Her efforts to expand the library's offerings and establish reading programs for children have had a significant impact on the town's literacy rates.
As you stroll through the charming town square, you'll be captivated by the beautiful murals adorning the walls. These masterpieces are the work of renowned artist, Isabella Torres, whose talent for capturing the essence of Riverside Grove has brought the town to life.
Riverside Grove's athletic achievements are also worth noting, thanks to former Olympic swimmer-turned-coach, Marcus Jenkins. Marcus has used his experience and passion to train the town's youth, leading the Riverside Grove Swim Team to several regional championships.
1. Oliver Hamilton [CHEF]
2. Elizabeth Chen [LIBRARIAN]
3. Isabella Torres [ARTIST]
4. Marcus Jenkins [COACH]

{story}
Ans:
"""

In [125]:
sampling_params = SamplingParams(temperature=0.1, top_p=0.95,max_tokens=100)
outputs = llm.generate(prompt, sampling_params)

result = outputs[0].outputs[0].text
print(result)

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  1.30it/s]

1. Ada Zhang [AI DEVELOPER]
2. Jayden Carter [CYBERPILOT]
3. Sentient AI [AI]
4. Ancient AI Guardian [AI]
